In [285]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import regularizers

import re

In [286]:
from keras import backend as K
K.clear_session()

In [287]:
physical_devices = tf.config.list_physical_devices()
CLASSES = 48
IMAGE_SIZE = 24

print(physical_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [288]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

## Building Dataset

### Attributes description

- a1 - face containing flag: (1-with face, 0-without face),
- a2 - image number in current class (person) beginning from 0,
- a3 - class (person) number beginning from 0,
- a4 - sex (0 - woman, 1 - man)
- a5 - race (0- white, 1 - negro, 2 - indian, ...)
- a6 - age (0 - baby, 1 - young, 2 - middle-age, 3 - old) 
- a7 - binokulars (0 - without, 1 - transparent, 2 - dark)
- a8 - emotional expression (not state!) (0 - sad, 1 - neutral, 2 - happy)

In [289]:
def read_file(filename):
  
  with open(filename, 'r') as file:
    lines = file.readlines()
    
    # Remove newLines
    for i, line in enumerate(lines):
      lines[i] = line.replace('\n', '')
    
    # We assume these are integers
    EXAMPLES_NR = int(lines[0])
    PIXELS_NR = int(lines[1])
    
    inputs = list()
    labels = np.zeros(EXAMPLES_NR, dtype=int)
    
    examples_raw = lines[2:EXAMPLES_NR+2]
    
    for i, example_raw in enumerate(examples_raw):
      # Split by spaces (treats multiple as one)
      tokens = re.split('\s+', example_raw)
      
      pixel_values = np.array(tokens[0:PIXELS_NR])
      attributes = tokens[PIXELS_NR:]
      
      pixel_values = np.array(pixel_values, dtype=float)
      pixel_values = pixel_values.reshape([IMAGE_SIZE, IMAGE_SIZE])
      
      inputs.append(pixel_values)
      labels[i] = int(attributes[2])
      
    inputs = np.array(inputs)
  return inputs, labels

In [290]:
# classes 0-15
X_0, y_0 = read_file('./data/x24x24.txt')
# classes 16-31
X_1, y_1 = read_file('./data/y24x24.txt')
# # classes 32-48
X_2, y_2 = read_file('./data/z24x24.txt')

print(len(X_0), len(y_0))
print(len(X_1), len(y_1))
print(len(X_2), len(y_2))

2321 2321
2208 2208
2306 2306


In [291]:
from sklearn.model_selection import train_test_split

# Concatenate train and test images
X = np.concatenate((X_0, X_1, X_2))
y = np.concatenate((y_0, y_1, y_2))
print(len(X), len(y))

N_TRAIN_EXAMPLES=int(len(X) * 0.8)
N_TEST_EXAMPLES=len(X) - N_TRAIN_EXAMPLES


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=N_TRAIN_EXAMPLES, test_size=N_TEST_EXAMPLES, random_state=1)

# One-Hot encoding
# Getting dummy variables
y_train_fixed = np.zeros((y_train.shape[0], CLASSES))
# y_valid_fixed = np.zeros((y_valid.shape[0], 10))
y_test_fixed = np.zeros((y_test.shape[0], CLASSES))

for i, value in enumerate(y_train):
  y_train_fixed[i][value] = 1
  
for i, value in enumerate(y_test):
  y_test_fixed[i][value] = 1

6835 6835


In [292]:
print(len(X_test))

1367


## Building a model

I decided to down-scale the alexNet as it was for the long time the SOTA Convolutional Neural Network Architecture

Full AlexNet implementation in keras:

```py
  keras.Input(shape=(227, 227, 3)),
  layers.Conv2D(96, kernel_size=11, strides=4, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(256, kernel_size=5, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(256, kernel_size=3, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dense(CLASSES, activation="softmax")
```

In [300]:
from keras import Sequential, Input

conv_regularizer = regularizers.l2(l=0.002)
dense_regularizer = regularizers.l2(l=0.03)

model = keras.Sequential(
  [
    Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    layers.Conv2D(24, kernel_size=3, strides=1, padding='same', kernel_regularizer=conv_regularizer),
    layers.LeakyReLU(),
    layers.Conv2D(64, kernel_size=5, padding='same', kernel_regularizer=conv_regularizer),
    
    layers.MaxPooling2D(pool_size=3, strides=2),
    
    layers.LeakyReLU(),
    layers.Conv2D(96, kernel_size=3, padding='same', kernel_regularizer=conv_regularizer),
    layers.LeakyReLU(),
    layers.Dropout(0.3),
    layers.Conv2D(96, kernel_size=3, strides=2, padding='same', kernel_regularizer=conv_regularizer),
    layers.Dropout(0.4),
    layers.LeakyReLU(),
    layers.Conv2D(64, kernel_size=3, padding='same', kernel_regularizer=conv_regularizer),
    
    layers.MaxPooling2D(pool_size=3, strides=2),
    
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.LeakyReLU(),
    layers.Dense(256, kernel_regularizer=dense_regularizer),
    layers.Dense(CLASSES, activation='softmax'),
  ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 24, 24, 24)        240       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 24, 24, 24)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 24, 24, 64)        38464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 64)       0         
 2D)                                                             
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 11, 11, 64)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 96)        55392     
                                                      

In [301]:
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy

# optimizer = Adam(learning_rate=0.00025)
optimizer = Adam(learning_rate=0.001)

model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=["accuracy"])

## Training Preparation

In [302]:
CHECKPOINT_PATH = 'models/checkpoints/famous48_test_model_v1_best.keras'

In [303]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
  filepath=CHECKPOINT_PATH,
  save_weights_only=False,
  monitor='val_accuracy',
  mode='max',
  save_best_only=True,
)

In [304]:
%%time
batch_size = 200
epochs = 100

history = model.fit(
  X_train, y_train_fixed,
  batch_size=batch_size,
  epochs=epochs,
  validation_split=0.2,
  shuffle=True,
  callbacks=[model_checkpoint_callback]
)

Epoch 1/100
22/22 [==============================] - 1s 34ms/step - loss: 10.1423 - accuracy: 0.0265 - val_loss: 8.2669 - val_accuracy: 0.0201
Epoch 2/100
22/22 [==============================] - 1s 25ms/step - loss: 7.0789 - accuracy: 0.0354 - val_loss: 6.0340 - val_accuracy: 0.0238
Epoch 3/100
22/22 [==============================] - 1s 26ms/step - loss: 5.4022 - accuracy: 0.0304 - val_loss: 4.8760 - val_accuracy: 0.0274
Epoch 4/100
22/22 [==============================] - 1s 26ms/step - loss: 4.5608 - accuracy: 0.0295 - val_loss: 4.3213 - val_accuracy: 0.0393
Epoch 5/100
22/22 [==============================] - 1s 24ms/step - loss: 4.1618 - accuracy: 0.0348 - val_loss: 4.0579 - val_accuracy: 0.0347
Epoch 6/100
22/22 [==============================] - 1s 24ms/step - loss: 3.9631 - accuracy: 0.0476 - val_loss: 3.9296 - val_accuracy: 0.0356
Epoch 7/100
22/22 [==============================] - 1s 26ms/step - loss: 3.6567 - accuracy: 0.0878 - val_loss: 3.4728 - val_accuracy: 0.1143
Epoch

In [306]:
model.evaluate(X_test, y_test_fixed)

43/43 [==============================] - 1s 10ms/step - loss: 1.2303 - accuracy: 0.8252


[1.2303251028060913, 0.8251646161079407]